In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Multi_Linear_Regression_Impletation").getOrCreate()

In [2]:
spark

In [5]:
df=spark.read.csv("tips.csv", header=True, inferSchema=True)

In [6]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [8]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [10]:
df.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [12]:
## Handling Categorical Features

from pyspark.ml.feature import StringIndexer

## StringIndexer helps to convert categorical features to numerical features

In [16]:
indexer=StringIndexer(inputCols=["sex", 'smoker', 'day', "time"], outputCols=["sex_indexed", 'smoker_indexed', 'day_indexed', 'time_indexed'])
df_r=indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [17]:
df_r.columns

['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [18]:
from pyspark.ml.feature import VectorAssembler
feature_assembler=VectorAssembler(inputCols=['tip', "size", 'sex_indexed', 'smoker_indexed', 'day_indexed', 'time_indexed'], outputCol="Independent_Features")
output=feature_assembler.transform(df_r)

In [19]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent_Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|S

In [26]:
finalized_data=output.select(['Independent_Features', "total_bill"])

In [27]:
finalized_data.show()

+--------------------+----------+
|Independent_Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [29]:
from pyspark.ml.regression import LinearRegression
## Train Test split
train_data, test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent_Features', labelCol='total_bill')
regressor=regressor.fit(train_data)

In [30]:
regressor.coefficients

DenseVector([3.2027, 2.518, -1.623, 1.8159, 0.0968, -0.7292])

In [31]:
regressor.intercept

3.476936782780916

In [32]:
## Predictions
pred_results=regressor.evaluate(test_data)

In [33]:
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent_Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.51|12.516267806821388|
| (6,[0,1],[2.0,2.0])|     12.69|14.918313554901898|
|(6,[0,1],[2.34,4.0])|     17.81|21.043162404604686|
|(6,[0,1],[3.39,2.0])|     11.61| 19.37010500801111|
| (6,[0,1],[3.6,3.0])|     24.06|22.560638539426797|
|(6,[0,1],[3.76,2.0])|     18.24| 20.55511424373082|
| (6,[0,1],[4.3,2.0])|      21.7|22.284587182348787|
| (6,[0,1],[9.0,4.0])|     48.33|  42.3733286475596|
|[1.1,2.0,1.0,1.0,...|      12.9|12.228724137797835|
|[1.25,2.0,1.0,0.0...|      8.51|10.357699527738347|
|[1.44,2.0,0.0,0.0...|      7.56|12.589236055091675|
|[1.5,2.0,1.0,0.0,...|     11.17|11.158381443765183|
|[1.61,2.0,1.0,1.0...|     10.59| 13.86211524649258|
|[1.64,2.0,0.0,1.0...|     15.36|15.581215347588735|
|[1.73,2.0,0.0,0.0...|      9.78|13.518027077682806|
|[1.92,1.0,0.0,1.0...|      8.58|13.5212965095

In [38]:
## Performance Metrics

pred_results.r2, pred_results.meanAbsoluteError, pred_results.meanSquaredError

(0.5483726285344201, 5.324424385822947, 51.0923649028309)